In [1]:
pip install yfinance


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import yfinance as yf
import pandas as pd

stock_symbols = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "JPM", "V", "DIS",
    "KO", "PEP", "NFLX", "INTC", "AMD", "BA", "CSCO", "XOM", "WMT", "PG",
    "T", "PFE", "MRK", "CVX", "TMO", "ADBE", "CRM", "ORCL", "IBM", "QCOM"
]
start_date = "2018-01-01"
end_date = "2023-12-31"

data = yf.download(stock_symbols, start=start_date, end=end_date, interval="1d")["Adj Close"]

sample_data = data.head(10)

data.to_csv("portfolio_stock_prices.csv")

print(sample_data)


[*********************100%***********************]  30 of 30 completed


Ticker           AAPL        ADBE    AMD       AMZN          BA         CRM  \
Date                                                                          
2018-01-02  40.524345  177.699997  10.98  59.450500  282.886383  103.948540   
2018-01-03  40.517296  181.039993  11.55  60.209999  283.801270  104.824646   
2018-01-04  40.705486  183.220001  12.12  60.479500  282.724426  106.208504   
2018-01-05  41.168938  185.339996  11.88  61.457001  294.322327  107.622231   
2018-01-08  41.016037  185.039993  12.28  62.343498  295.570740  108.378868   
2018-01-09  41.011318  186.699997  11.82  62.634998  303.461487  108.667587   
2018-01-10  41.001904  187.110001  11.96  62.716499  305.205505  108.319138   
2018-01-11  41.234802  188.919998  12.14  63.834000  312.695984  108.617805   
2018-01-12  41.660606  195.050003  12.02  65.260002  320.405731  109.752769   
2018-01-16  41.448883  192.460007  11.91  65.242996  319.405090  108.408737   

Ticker           CSCO        CVX         DIS      G

In [3]:
import pandas as pd

data = pd.read_csv("portfolio_stock_prices.csv", index_col=0, parse_dates=True)

daily_returns = data.pct_change()

daily_returns.to_csv("portfolio_daily_returns.csv")

print(daily_returns.head(10))


                AAPL      ADBE       AMD      AMZN        BA       CRM  \
Date                                                                     
2018-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
2018-01-03 -0.000174  0.018796  0.051913  0.012775  0.003234  0.008428   
2018-01-04  0.004645  0.012042  0.049351  0.004476 -0.003794  0.013202   
2018-01-05  0.011385  0.011571 -0.019802  0.016163  0.041022  0.013311   
2018-01-08 -0.003714 -0.001619  0.033670  0.014425  0.004242  0.007030   
2018-01-09 -0.000115  0.008971 -0.037459  0.004676  0.026697  0.002664   
2018-01-10 -0.000230  0.002196  0.011844  0.001301  0.005747 -0.003207   
2018-01-11  0.005680  0.009673  0.015050  0.017818  0.024542  0.002757   
2018-01-12  0.010326  0.032448 -0.009885  0.022339  0.024656  0.010449   
2018-01-16 -0.005082 -0.013279 -0.009151 -0.000261 -0.003123 -0.012246   

                CSCO       CVX       DIS     GOOGL  ...       PEP       PFE  \
Date                            

In [4]:
import pandas as pd

data = pd.read_csv("portfolio_stock_prices.csv", index_col=0, parse_dates=True)

short_window = 10
long_window = 50

long_positions = {}
short_positions = {}
portfolio_returns = []

for stock in data.columns:
    stock_data = data[stock].dropna()  # Drop missing values
    
    # Calculate SMAs
    short_sma = stock_data.rolling(window=short_window).mean()
    long_sma = stock_data.rolling(window=long_window).mean()
    
    # Generate trading signals
    signals = pd.Series(0, index=stock_data.index)
    signals[short_sma > long_sma] = 1  # Long position
    signals[short_sma < long_sma] = -1  # Short position
    
    # Calculate daily returns
    daily_returns = stock_data.pct_change()
    
    # Calculate long and short returns
    long_returns = daily_returns[signals.shift(1) == 1]
    short_returns = -daily_returns[signals.shift(1) == -1]  # Shorting gains when prices drop
    
    long_positions[stock] = signals[signals == 1]
    short_positions[stock] = signals[signals == -1]
    
    portfolio_returns.append(daily_returns * signals.shift(1))

portfolio_returns_df = pd.concat(portfolio_returns, axis=1).mean(axis=1)

pd.DataFrame(long_positions).to_csv("long_positions.csv")
pd.DataFrame(short_positions).to_csv("short_positions.csv")

# Save portfolio returns
portfolio_returns_df.to_csv("long_short_portfolio_returns.csv")

print("Portfolio Returns (First 10 Days):")
print(portfolio_returns_df.head(10))


Portfolio Returns (First 10 Days):
Date
2018-01-02    NaN
2018-01-03    0.0
2018-01-04    0.0
2018-01-05    0.0
2018-01-08    0.0
2018-01-09    0.0
2018-01-10    0.0
2018-01-11    0.0
2018-01-12    0.0
2018-01-16    0.0
dtype: float64


In [5]:
import pandas as pd
import numpy as np

# Load portfolio returns
portfolio_returns = pd.read_csv("long_short_portfolio_returns.csv", index_col=0, parse_dates=True)

cumulative_returns = (1 + portfolio_returns).cumprod()

average_return = portfolio_returns.mean()  # Mean daily return
volatility = portfolio_returns.std()       # Standard deviation of daily returns
sharpe_ratio = average_return / volatility # Assuming 0% risk-free rate

rolling_max = cumulative_returns.cummax()
drawdown = (cumulative_returns - rolling_max) / rolling_max
max_drawdown = drawdown.min()

print("Performance Metrics (Portfolio Level):")
print(f"Average Daily Return: {average_return.mean():.5f}")
print(f"Return Volatility: {volatility.mean():.5f}")
print(f"Sharpe Ratio: {sharpe_ratio.mean():.5f}")
print(f"Maximum Drawdown: {max_drawdown.mean():.5%}")

print("\nPerformance Metrics (Individual Stocks):")
print(f"Average Daily Return per Stock:\n{average_return}")
print(f"Return Volatility per Stock:\n{volatility}")
print(f"Sharpe Ratio per Stock:\n{sharpe_ratio}")
print(f"Maximum Drawdown per Stock:\n{max_drawdown}")


Performance Metrics (Portfolio Level):
Average Daily Return: 0.00001
Return Volatility: 0.00997
Sharpe Ratio: 0.00115
Maximum Drawdown: -26.94956%

Performance Metrics (Individual Stocks):
Average Daily Return per Stock:
0    0.000011
dtype: float64
Return Volatility per Stock:
0    0.00997
dtype: float64
Sharpe Ratio per Stock:
0    0.001145
dtype: float64
Maximum Drawdown per Stock:
0   -0.269496
dtype: float64


In [6]:
import statsmodels.api as sm
import pandas as pd

# Load portfolio returns and market data
portfolio_returns = pd.read_csv("long_short_portfolio_returns.csv", index_col=0, parse_dates=True)

market_data = yf.download("SPY", start=portfolio_returns.index[0], end=portfolio_returns.index[-1], interval="1d")["Adj Close"]
market_returns = market_data.pct_change()

aligned_data = pd.concat([portfolio_returns, market_returns], axis=1).dropna()
aligned_data.columns = ["Strategy Returns", "Market Returns"]

X = sm.add_constant(aligned_data["Market Returns"])  # Add constant for intercept
y = aligned_data["Strategy Returns"]
model = sm.OLS(y, X).fit()

print(model.summary())


[*********************100%***********************]  1 of 1 completed


                            OLS Regression Results                            
Dep. Variable:       Strategy Returns   R-squared:                       0.153
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     272.1
Date:                Mon, 09 Dec 2024   Prob (F-statistic):           2.51e-56
Time:                        14:46:46   Log-Likelihood:                 4931.5
No. Observations:                1507   AIC:                            -9859.
Df Residuals:                    1505   BIC:                            -9848.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.0002      0.000      0.

In [7]:

total_trades = (signals != 0).sum()  
correct_trades = ((signals.shift(1) * daily_returns) > 0).sum() 
accuracy = correct_trades / total_trades

print(f"Signal Accuracy: {accuracy:.2%}")


Signal Accuracy: 52.05%


In [8]:
import pandas as pd

portfolio_returns = pd.read_csv("long_short_portfolio_returns.csv", index_col=0, parse_dates=True)

if 'Strategy Returns' not in portfolio_returns.columns:
    portfolio_returns.columns = ['Strategy Returns']  # Rename the column if needed

portfolio_returns['Year'] = portfolio_returns.index.year

annual_performance = portfolio_returns.groupby('Year')['Strategy Returns'].agg([
    ('Average_Return', 'mean'),
    ('Volatility', 'std'),
    ('Sharpe_Ratio', lambda x: x.mean() / x.std() if x.std() != 0 else 0)
]).reset_index()

print("Annual Performance Metrics:")
print(annual_performance)


Annual Performance Metrics:
   Year  Average_Return  Volatility  Sharpe_Ratio
0  2018       -0.000015    0.007167     -0.002057
1  2019        0.000120    0.006029      0.019832
2  2020        0.000427    0.017839      0.023922
3  2021       -0.000128    0.004648     -0.027624
4  2022       -0.000283    0.011634     -0.024327
5  2023       -0.000055    0.005789     -0.009534


In [9]:
import numpy as np
import pandas as pd

portfolio_ema_returns = pd.read_csv("long_short_portfolio_returns.csv", index_col=0, parse_dates=True)

cumulative_returns = (1 + portfolio_ema_returns).cumprod()

mean_return = portfolio_ema_returns.mean().mean()  # Mean return across the portfolio
volatility = portfolio_ema_returns.std().mean()    # Standard deviation of returns
downside_deviation = portfolio_ema_returns[portfolio_ema_returns < 0].std().mean()  # Downside deviation
max_drawdown = ((cumulative_returns / cumulative_returns.cummax()) - 1).min().min()

sharpe_ratio = mean_return / volatility
sortino_ratio = mean_return / downside_deviation
calmar_ratio = mean_return / abs(max_drawdown)

print("Performance Metrics:")
print(f"Sharpe Ratio: {sharpe_ratio:.5f}")
print(f"Sortino Ratio: {sortino_ratio:.5f}")
print(f"Calmar Ratio: {calmar_ratio:.5f}")


Performance Metrics:
Sharpe Ratio: 0.00115
Sortino Ratio: 0.00128
Calmar Ratio: 0.00004
